# LangChain の記法解説 (LCEL)


In [ ]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env", override=True)

## Runnable と RunnableSequence


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

output_parser = StrOutputParser()

In [ ]:
prompt_value = prompt.invoke({"dish": "カレー"})
ai_message = model.invoke(prompt_value)
output = output_parser.invoke(ai_message)
print(output)

In [ ]:
chain = prompt | model | output_parser

output = chain.invoke({"dish": "カレー"})
print(output)

### Runnable の実行方法―invoke・stream・batch


In [ ]:
chain = prompt | model | output_parser

for chunk in chain.stream({"dish": "カレー"}):
    print(chunk, end="", flush=True)

In [ ]:
chain = prompt | model | output_parser

inputs = [{"dish": "カレー"}, {"dish": "うどん"}]
outputs = chain.batch(inputs)

for i, o in zip(inputs, outputs):
    print(f"input: {i}")
    print(f"output: {o[:30]}...")

### LCEL の「|」で様々な Runnable を連鎖させる


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

output_parser = StrOutputParser()

In [ ]:
cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーの質問にステップバイステップで回答してください。"),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | model | output_parser

In [ ]:
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ステップバイステップで考えた回答から結論だけ抽出してください。"),
        ("human", "{text}"),
    ]
)

summarize_chain = summarize_prompt | model | output_parser

In [ ]:
cot_summarize_chain = cot_chain | summarize_chain
output = cot_summarize_chain.invoke({"question": "10 + 2 * 3"})
print(output)

In [ ]:
# このセルのコードでは、LCELで記述したチェーンを可視化します。
# mermaid.inkのサービスを利用しているため以下のエラーが発生する場合がありますが、エラーになった場合でも続きのハンズオンには影響はありません。
# ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

from IPython.display import Image, display

display(Image(cot_summarize_chain.get_graph().draw_mermaid_png()))

## RunnableLambda―任意の関数を Runnable にする


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}"),
    ]
)

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

output_parser = StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnableLambda


def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | RunnableLambda(upper)

output = chain.invoke({"input": "Hello!"})
print(output)

### RunnableLambda への自動変換


In [ ]:
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | upper
output = chain.invoke({"input": "Hello!"})
print(output)

### Runnable の入力の型と出力の型に注意


In [ ]:
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | upper

# 以下のコードを実行するとエラーになります
# output = chain.invoke({"input": "Hello!"})

In [ ]:
chain = prompt | model | StrOutputParser() | upper

output = chain.invoke({"input": "Hello!"})
print(output)

## RunnableParallel―複数の Runnable を並列で処理する


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
output_parser = StrOutputParser()

In [ ]:
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは楽観主義者です。ユーザーの入力に対して楽観的な意見をください。",
        ),
        ("human", "{topic}"),
    ]
)
optimistic_chain = optimistic_prompt | model | output_parser

In [ ]:
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見をください。",
        ),
        ("human", "{topic}"),
    ]
)
pessimistic_chain = pessimistic_prompt | model | output_parser

In [ ]:
import pprint
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel(
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
)

output = parallel_chain.invoke({"topic": "生成AIの進化について"})
pprint.pprint(output)

In [ ]:
from IPython.display import Image, display

display(Image(parallel_chain.get_graph().draw_mermaid_png()))

### RunnableParallel の出力を Runnable の入力に連結する


In [ ]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的AIです。2つの意見をまとめてください。"),
        (
            "human",
            "楽観的意見: {optimistic_opinion}\n悲観的意見: {pessimistic_opinion}",
        ),
    ]
)

In [ ]:
from langchain_core.runnables import RunnableParallel

synthesize_chain = (
    RunnableParallel(
        {
            "optimistic_opinion": optimistic_chain,
            "pessimistic_opinion": pessimistic_chain,
        }
    )
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

In [ ]:
from IPython.display import Image, display

display(Image(synthesize_chain.get_graph().draw_mermaid_png()))

### RunnableParallel への自動変換


In [ ]:
synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
    | synthesize_prompt
    | model
    | output_parser
)

In [ ]:
output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

## RunnablePassthrough―入力をそのまま出力する


In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = DirectoryLoader(
    path="../tmp/langchain",
    glob="**/*.mdx",
    loader_cls=TextLoader,
)
raw_docs = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(raw_docs)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(docs, embeddings)

retriever = db.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model_name="gpt-4.1-nano", temperature=0)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke("LangGraphとは")
print(output)

### assign―RunnableParallel に値を追加する


In [ ]:
import pprint

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | RunnablePassthrough.assign(answer=prompt | model | StrOutputParser())

output = chain.invoke("LangGraphとは")
pprint.pprint(output)

### LCEL を使わない実装


In [ ]:
def invoke_rag(query: str) -> str:
    documents = retriever.invoke(query)
    chain = prompt | model | StrOutputParser()
    return chain.invoke({"question": query, "context": documents})


output = invoke_rag("LangGraphとは")
print(output)

In [ ]:
from langsmith import traceable


@traceable
def invoke_rag(query: str) -> str:
    documents = retriever.invoke(query)
    chain = prompt | model | StrOutputParser()
    return chain.invoke({"question": query, "context": documents})


output = invoke_rag("LangGraphとは")
print(output)
